In [1]:
# -*- coding:utf-8 -*-

import plotly.plotly as py
import plotly.graph_objs as go
import json
import pandas as pd
import datetime

#plotly.tools.set_credentials_file(username='lyy901207', api_key='Rg5JLLHl338IQ5AwnjdC')


In [2]:
# delete file
import requests
from requests.auth import HTTPBasicAuth
username = 'lyy901207' # Replace with YOUR USERNAME
api_key = 'Rg5JLLHl338IQ5AwnjdC' # Replace with YOUR API KEY

auth = HTTPBasicAuth(username, api_key)
headers = {'Plotly-Client-Platform': 'python'}

#fid_permanent_delete = 'lyy901207'+':72'
#requests.post('https://api.plot.ly/v2/files/'+fid_permanent_delete+'/trash', auth=auth, headers=headers)
#requests.delete('https://api.plot.ly/v2/files/'+fid_permanent_delete+'/permanent_delete', auth=auth, headers=headers)

In [3]:
def get_pages(username, page_size):
    url = 'https://api.plot.ly/v2/folders/all?user='+username+'&page_size='+str(page_size)
    response = requests.get(url, auth=auth, headers=headers)
    if response.status_code != 200:
        return
    page = json.loads(response.content)
    yield page
    while True:
        resource = page['children']['next']
        if not resource:
            break
        response = requests.get(resource, auth=auth, headers=headers)
        if response.status_code != 200:
            break
        page = json.loads(response.content)
        yield page

def permanently_delete_files(username, page_size=500, filetype_to_delete='plot'):
    for page in get_pages(username, page_size):
        for x in range(0, len(page['children']['results'])):
            fid = page['children']['results'][x]['fid']
            res = requests.get('https://api.plot.ly/v2/files/' + fid, auth=auth, headers=headers)
            res.raise_for_status()
            if res.status_code == 200:
                json_res = json.loads(res.content)
                if json_res['filetype'] == filetype_to_delete:
                    # move to trash
                    requests.post('https://api.plot.ly/v2/files/'+fid+'/trash', auth=auth, headers=headers)
                    # permanently delete
                    requests.delete('https://api.plot.ly/v2/files/'+fid+'/permanent_delete', auth=auth, headers=headers)

permanently_delete_files(username, filetype_to_delete='plot')
permanently_delete_files(username, filetype_to_delete='grid')

#  formatting dataset

In [4]:
def load_data(path):
    with open(path, 'r') as f:
        data = json.load(f)
    return data
    
def extract_data(data, subdata, variable):
    container = {}
    for network in data['networks']:
        container[network['ts']]={}
        for varid in network[subdata].keys():
            value = network[subdata][varid][variable]
            container[network['ts']][str(varid)]=value
    return container

# write the result in CSV format
def write_csv(data, path):
    '''
    This function is to write extracted dataset to csv file
    data: formating in pandas dataframe
    '''
    data.to_csv(path)

ModuleNotFoundError: No module named 'util'

# {'id': lane.id, 'type': lane.type, 'link_id': lane.link.id,
        'freeSpeed': lane.freeSpeed, 'freeTravelTime': lane.freeTravelTime, 'fixedCharge': lane.fixedCharge,
        'speed': lane.speed, 'countPcu': lane.countPcu, 'density': lane.density, 'travelTime': lane.travelTime,
        'charge': lane.charge}

In [27]:
# extract lane speed data
#path = 'C:/Users/lyy90/OneDrive/Documents/GitHub/meso_v2.0/visualization/'
path = "F:/meso_v2.0/visualization/"
uniform_1_f = ['uniform_vol_sim1_False_0_output.json', 'uniform_vol_sim1_False_1_output.json', 'uniform_vol_sim1_False_2_output.json']
uniform_2_f = ['uniform_vol_sim2_False_0_output.json', 'uniform_vol_sim2_False_1_output.json', 'uniform_vol_sim2_False_2_output.json', 
               'uniform_vol_sim2_False_3_output.json', 'uniform_vol_sim2_False_4_output.json',]
uniform_3_f = ['uniform_vol_sim3_False_0_output.json', 'uniform_vol_sim3_False_1_output.json', 'uniform_vol_sim3_False_2_output.json',
                  'uniform_vol_sim3_False_3_output.json', 'uniform_vol_sim3_False_4_output.json', 'uniform_vol_sim3_False_5_output.json',
                  'uniform_vol_sim3_False_6_output.json']

uniform_1_t = ['uniform_vol_sim1_True_0_output.json', 'uniform_vol_sim1_True_1_output.json', 'uniform_vol_sim1_True_2_output.json']
uniform_2_t = ['uniform_vol_sim2_True_0_output.json', 'uniform_vol_sim2_True_1_output.json', 'uniform_vol_sim2_True_2_output.json']
uniform_3_t = ['uniform_vol_sim3_True_0_output.json', 'uniform_vol_sim3_True_1_output.json', 'uniform_vol_sim3_True_2_output.json',
               'uniform_vol_sim3_True_3_output.json', 'uniform_vol_sim3_True_4_output.json', 'uniform_vol_sim3_True_5_output.json',
               'uniform_vol_sim3_True_6_output.json', 'uniform_vol_sim3_True_7_output.json']

normal_1_f=["normal_whole_vol_sim1_False_0_output.json","normal_whole_vol_sim1_False_1_output.json","normal_whole_vol_sim1_False_2_output.json"
           ]
normal_2_f=["normal_whole_vol_sim2_False_0_output.json", "normal_whole_vol_sim2_False_1_output.json", "normal_whole_vol_sim2_False_2_output.json",
            "normal_whole_vol_sim2_False_3_output.json", "normal_whole_vol_sim2_False_4_output.json", "normal_whole_vol_sim2_False_5_output.json"]
normal_3_f=["normal_whole_vol_sim3_False_0_output.json", "normal_whole_vol_sim3_False_1_output.json", "normal_whole_vol_sim3_False_2_output.json",
            "normal_whole_vol_sim3_False_3_output.json", "normal_whole_vol_sim3_False_4_output.json", "normal_whole_vol_sim3_False_5_output.json",
            "normal_whole_vol_sim3_False_6_output.json"]

normal_1_t=["normal_whole_vol_sim1_True_0_output.json", "normal_whole_vol_sim1_True_1_output.json", "normal_whole_vol_sim1_True_2_output.json"]
normal_2_t=["normal_whole_vol_sim2_True_0_output.json", "normal_whole_vol_sim2_True_1_output.json", "normal_whole_vol_sim2_True_2_output.json",
            "normal_whole_vol_sim2_True_3_output.json", "normal_whole_vol_sim2_True_4_output.json"]
normal_3_t=["normal_whole_vol_sim3_True_0_output.json", "normal_whole_vol_sim3_True_1_output.json", "normal_whole_vol_sim3_True_2_output.json",
            "normal_whole_vol_sim3_True_3_output.json", "normal_whole_vol_sim3_True_4_output.json", "normal_whole_vol_sim3_True_5_output.json",
            "normal_whole_vol_sim3_True_6_output.json"]

random_1_f = ["random_vol_sim1_False_0_output.json", "random_vol_sim1_False_1_output.json", "random_vol_sim1_False_2_output.json"]
random_2_f = ["random_vol_sim2_False_0_output.json", "random_vol_sim2_False_1_output.json", "random_vol_sim2_False_2_output.json",
              "random_vol_sim2_False_3_output.json", "random_vol_sim2_False_4_output.json"]
random_3_f = ["random_vol_sim3_False_0_output.json", "random_vol_sim3_False_1_output.json", "random_vol_sim3_False_2_output.json",
              "random_vol_sim3_False_3_output.json", "random_vol_sim3_False_4_output.json", "random_vol_sim3_False_5_output.json",
              "random_vol_sim3_False_6_output.json"]

random_1_t = ["random_vol_sim1_True_0_output.json", "random_vol_sim1_True_1_output.json", "random_vol_sim1_True_2_output.json"]
random_2_t = ["random_vol_sim2_True_0_output.json", "random_vol_sim2_True_1_output.json", "random_vol_sim2_True_2_output.json",
              "random_vol_sim2_True_3_output.json", "random_vol_sim2_True_4_output.json"]
random_3_t = ["random_vol_sim3_True_0_output.json", "random_vol_sim3_True_1_output.json", "random_vol_sim3_True_2_output.json",
              "random_vol_sim3_True_3_output.json", "random_vol_sim3_True_4_output.json", "random_vol_sim3_True_5_output.json",
              "random_vol_sim3_True_6_output.json"]

In [6]:
# uniform sim
file_list1 = [path+name for name in uniform_1_f]
file_list2 = [path+name for name in uniform_2_f]
file_list3 = [path+name for name in uniform_3_f]

file_list1_t = [path+name for name in uniform_1_t]
file_list2_t = [path+name for name in uniform_2_t]
file_list3_t = [path+name for name in uniform_3_t]

In [51]:
# normal sim
file_list1 = [path+name for name in normal_1_f]
file_list2 = [path+name for name in normal_2_f]
file_list3 = [path+name for name in normal_3_f]

file_list1_t = [path+name for name in normal_1_t]
file_list2_t = [path+name for name in normal_2_t]
file_list3_t = [path+name for name in normal_3_t]

In [28]:
# random sim
file_list1 = [path+name for name in random_1_f]
file_list2 = [path+name for name in random_2_f]
file_list3 = [path+name for name in random_3_f]

file_list1_t = [path+name for name in random_1_t]
file_list2_t = [path+name for name in random_2_t]
file_list3_t = [path+name for name in random_3_t]

In [88]:
#write_csv(df,'C:/Users/lyy90/OneDrive/Documents/GitHub/meso_v2.0/visualization/uniform_fix2_pcu.csv')


# plotting

#  计算每条道被占用的时间

In [29]:
def calculateOccupy(data):
    container = {}
    for network in data['networks']:
        container[network['ts']]={'high':0, 'low':0}
        for ve in network['vehicles'].keys():
            #print('lanetype is:', network['vehicles'][ve]['laneType'])
            if network['vehicles'][ve]['laneType']=='1':
                container[network['ts']]['high'] +=1
            if network['vehicles'][ve]['laneType']=='0':
                container[network['ts']]['low'] +=1
    return container

In [30]:
oc1 = []
oc2 = []
oc3 = []

for file in file_list1:
    data1 = load_data(file)
    temp_data1 = calculateOccupy(data1)
    #print(temp_data1)
    temp_data1 = pd.DataFrame(temp_data1).sort_index()
    temp_data1 = temp_data1.T
    oc1.append(temp_data1)
oc1 = pd.concat(oc1)

for file in file_list2:
    data2 = load_data(file)
    temp_data2 = calculateOccupy(data2)
    #print(temp_data1)
    temp_data2 = pd.DataFrame(temp_data2).sort_index()
    temp_data2 = temp_data2.T
    oc2.append(temp_data2)
oc2 = pd.concat(oc2)

for file in file_list3:
    data3 = load_data(file)
    temp_data3 = calculateOccupy(data3)
    #print(temp_data1)
    temp_data3 = pd.DataFrame(temp_data3).sort_index()
    temp_data3 = temp_data3.T
    oc3.append(temp_data3)
oc3 = pd.concat(oc3)

In [32]:
trace1 = go.Scatter(
    x = oc1.index - 1546293600,
    y = oc1['high']/(oc1["high"]+oc1["low"]),
    name = 'charge 2078',
    line = dict(
        width = 2)
)

trace2 = go.Scatter(
    x = oc2.index - 1546293600,
    y = oc2['high']/(oc2["high"]+oc2["low"]),
    name = 'charge 4156',
    line = dict(
        width = 2)
)

trace3 = go.Scatter(
    x = oc3.index - 1546293600,
    y = oc3['high']/(oc3["high"]+oc3["low"]),
    name = 'charge 6234',
    line = dict(
        width = 2)
)

# format
data = [trace1, trace2, trace3]

layout = go.Layout(
    title=go.layout.Title(
        text='Occupy Rate by Time Stamp Random',
        xref='paper',
        x=0.5
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='time stamp',
            font=dict(
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='count',
            font=dict(
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename = 'Occupy Rate by Time Stamp Random', auto_open=True)

'https://plot.ly/~lyy901207/120'

# 计算正在running的车辆总数

In [33]:
# 计算每辆车的行驶时间
def calculateVeRunTime(data, container):    
    for network in data['networks']:
        for ve in network['endVehicles'].keys():
            vid = network['endVehicles'][ve]['id']
            vid = int(vid)
            container[vid] = (network['endVehicles'][ve]['finishTs']  - 
                              network['endVehicles'][ve]['startTs'], network["endVehicles"][ve]["type"])


In [34]:
ve1 = {}

for file in file_list1:
    data1 = load_data(file)
    calculateVeRunTime(data1, ve1)

ve1 = pd.DataFrame.from_dict(ve1, orient='index', columns = ['travel_time', "type"]).sort_index()

In [35]:
ve2 = {}
for file in file_list2:
    data2 = load_data(file)
    calculateVeRunTime(data2, ve2)
ve2 = pd.DataFrame.from_dict(ve2, orient='index', columns = ['travel_time', "type"]).sort_index()    

In [36]:
ve3 = {}
for file in file_list3:
    data3 = load_data(file)
    calculateVeRunTime(data3, ve3)
ve3 = pd.DataFrame.from_dict(ve3, orient='index', columns = ['travel_time', "type"]).sort_index()    

In [37]:
ve4 = {}

for file in file_list1_t:
    data4 = load_data(file)
    calculateVeRunTime(data4, ve4)
ve4 = pd.DataFrame.from_dict(ve4, orient='index', columns = ['travel_time', "type"]).sort_index()    

In [38]:
ve5 = {}
for file in file_list2_t:
    data5 = load_data(file)
    calculateVeRunTime(data5, ve5)
ve5 = pd.DataFrame.from_dict(ve5, orient='index', columns = ['travel_time', "type"]).sort_index()    

In [39]:
ve6 = {}
for file in file_list3_t:
    data6 = load_data(file)
    calculateVeRunTime(data6, ve6)
ve6 = pd.DataFrame.from_dict(ve6, orient='index', columns = ['travel_time', "type"]).sort_index()    

In [40]:
import numpy as np

def calculateTimeSave(data1,data2):
    save_bus = data1.loc[data1["type"]=="bus"]['travel_time'] - data2.loc[data2["type"]=="bus"]['travel_time']
    save_car = data1.loc[data1["type"]=="car"]['travel_time'] - data2.loc[data2["type"]=="car"]['travel_time']
    ratio_bus = save_bus[save_bus<0].size/save_bus.size
    ratio_car = save_car[save_car<0].size/save_car.size
    return print("bus: ", ratio_bus, "; car: ", ratio_car)

def calculateSta(data1, data2):
    dt = data1['travel_time'] - data2['travel_time']
    m = np.mean(dt)
    std = np.std(dt)
    return print("mean: ", m, " ; std:", std)

calculateTimeSave(ve1, ve4)
calculateTimeSave(ve2, ve5)
calculateTimeSave(ve3, ve6)

calculateSta(ve1, ve4)
calculateSta(ve2, ve5)
calculateSta(ve3, ve6)

bus:  0.2916666666666667 ; car:  0.2318125770653514
bus:  0.35635964912280704 ; car:  0.3588162762022195
bus:  0.44298245614035087 ; car:  0.4231401561857789
mean:  95.49085659287776  ; std: 95.76754752995132
mean:  53.43383060635226  ; std: 448.51358850859464
mean:  10.95925569457812  ; std: 1355.382610127092


In [46]:
trace0 = go.Scatter(
     mode = "markers",
     x = ve1.loc[ve1["type"]=="bus"].index,
     y = ve1.loc[ve1["type"]=="bus"]['travel_time'] - ve4.loc[ve1["type"]=="bus"]['travel_time'],
     name = "bus",
     marker = dict(
         color = "#E6DA4E")
)

trace1 = go.Scatter(
     mode = "markers" ,
     x = ve1.loc[ve1["type"]=="car"].index,
     y = ve1.loc[ve1["type"]=="car"]['travel_time'] - ve4.loc[ve1["type"]=="car"]['travel_time'],
     name = "car",
     marker = dict(
         color = "#3A7331",
         symbol = "diamond-open" )
)

layout = go.Layout(
    title=go.layout.Title(
        text='Travel Time Difference (2078) Random',
        xref='paper',
        x=0.5
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='vehicle ID',
            font=dict(
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='travel time diff (sec)',
            font=dict(
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


# format
data = [trace0, trace1]

fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename='Travel Time Difference (2078) Random', auto_open=True)

'https://plot.ly/~lyy901207/122'

In [47]:
trace2 = go.Scatter(
     mode = "markers",
     x = ve2.loc[ve2["type"]=="bus"].index,
     y = ve2.loc[ve2["type"]=="bus"]['travel_time'] - ve5.loc[ve5["type"]=="bus"]['travel_time'],
     name = "bus",
     marker = dict(
         color = "#E6DA4E")
)

trace3 = go.Scatter(
     mode = "markers" ,
     x = ve2.loc[ve2["type"]=="car"].index,
     y = ve2.loc[ve2["type"]=="car"]['travel_time'] - ve5.loc[ve5["type"]=="car"]['travel_time'],
     name = "car",
     marker = dict(
         color = "#3A7331",
         symbol = "diamond-open" )
)

layout = go.Layout(
    title=go.layout.Title(
        text='Travel Time Difference (4156) Random',
        xref='paper',
        x=0.5
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='vehicle ID',
            font=dict(
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='travel time diff (sec)',
            font=dict(
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


# format
data = [trace2, trace3]

fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename='Travel Time Difference (4156) Random', auto_open=True)

'https://plot.ly/~lyy901207/124'

In [48]:
trace4 = go.Scatter(
     mode = "markers",
     x = ve3.loc[ve3["type"]=="bus"].index,
     y = ve3.loc[ve3["type"]=="bus"]['travel_time'] - ve6.loc[ve6["type"]=="bus"]['travel_time'],
     name = "bus",
     marker = dict(
         color = "#E6DA4E")
)

trace5 = go.Scatter(
     mode = "markers" ,
     x = ve3.loc[ve3["type"]=="car"].index,
     y = ve3.loc[ve3["type"]=="car"]['travel_time'] - ve6.loc[ve6["type"]=="car"]['travel_time'],
     name = "car",
     marker = dict(
         color = "#3A7331",
         symbol = "diamond-open" )
)

layout = go.Layout(
    title=go.layout.Title(
        text='Travel Time Difference (6234) Random',
        xref='paper',
        x=0.5
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='vehicle ID',
            font=dict(
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='travel time diff (sec)',
            font=dict(
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


# format
data = [trace4, trace5]

fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename='Travel Time Difference (6234) Random', auto_open=True)

'https://plot.ly/~lyy901207/126'

In [ ]:
# vehicle

In [49]:
# 计算每辆车的数据
def calculateVehicle(data):
    container = {}
    for network in data['networks']:
        container[network['ts']] = {'bus':0, 'car':0}
        for ve in network['vehicles'].keys():
            if network['vehicles'][ve]['type'] == 'bus':
                container[network['ts']]['bus'] += 1
            else:
                container[network['ts']]['car'] += 1
    return container



In [22]:
count_ve1 = []
count_ve2 = []
count_ve3 = []

for file in file_list1:
    data1 = load_data(file)
    temp_data1 = calculateVehicle(data1)
    #print(temp_data1)
    temp_data1 = pd.DataFrame(temp_data1).sort_index()
    temp_data1 = temp_data1.T
    count_ve1.append(temp_data1)
count_ve1 = pd.concat(count_ve1)

for file in file_list2:
    data2 = load_data(file)
    temp_data2 = calculateVehicle(data2)
    #print(temp_data1)
    temp_data2 = pd.DataFrame(temp_data2).sort_index()
    temp_data2 = temp_data2.T
    count_ve2.append(temp_data2)
count_ve2 = pd.concat(count_ve2)

for file in file_list3:
    data3 = load_data(file)
    temp_data3 = calculateVehicle(data3)
    #print(temp_data1)
    temp_data3 = pd.DataFrame(temp_data3).sort_index()
    temp_data3 = temp_data3.T
    count_ve3.append(temp_data3)
count_ve3 = pd.concat(count_ve3)

In [51]:
trace1 = go.Scatter(
    x = count_ve1.index - 1546293600,
    y = count_ve1['bus'],
    name = 'bus 1',
    line = dict(
        width = 2)
)

trace2 = go.Scatter(
    x = count_ve2.index - 1546293600,
    y = count_ve2['bus'],
    name = 'bus 2',
    line = dict(
        width = 2)
)

trace3 = go.Scatter(
    x = count_ve3.index - 1546293600,
    y = count_ve3['bus'],
    name = 'bus 3',
    line = dict(
        width = 2)
)


trace4 = go.Scatter(
    x = count_ve1.index - 1546293600,
    y = count_ve1['car'],
    name = 'car 1',
    line = dict(
        width = 2, dash = "dot")
)

trace5 = go.Scatter(
    x = count_ve2.index - 1546293600,
    y = count_ve2['car'],
    name = 'car 2',
    line = dict(
        width = 2, dash = "dot")
)

trace6 = go.Scatter(
    x = count_ve3.index - 1546293600,
    y = count_ve3['car'],
    name = 'car 3',
    line = dict(
        width = 2, dash = "dot")
)

# format
data = [trace1, trace2, trace3, trace4, trace5, trace6]

layout = go.Layout(
    title=go.layout.Title(
        text='Vehicle Number by Time Stamp Random',
        xref='paper',
        x=0.5
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='time stamp',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='count',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename = 'Vehicle Number by Time Stamp Random', auto_open=True)

'https://plot.ly/~lyy901207/128'

In [255]:
df1 = []
df2 = []
df3 = []
for file in file_list1:
    data1 = load_data(file)
    temp_data1 = calculateOccupy(data1)
    #print(temp_data1)
    temp_data1 = pd.DataFrame(temp_data1).sort_index()
    temp_data1 = temp_data1.T
    df1.append(temp_data1)
df1 = pd.concat(df1)

for file in file_list2:
    data2 = load_data(file)
    temp_data2 = calculateOccupy(data2)
    #print(temp_data1)
    temp_data2 = pd.DataFrame(temp_data2).sort_index()
    temp_data2 = temp_data2.T
    df2.append(temp_data2)
df2 = pd.concat(df2)

for file in file_list3:
    data3 = load_data(file)
    temp_data3 = calculateOccupy(data3)
    #print(temp_data1)
    temp_data3 = pd.DataFrame(temp_data3).sort_index()
    temp_data3 = temp_data3.T
    df3.append(temp_data3)
df3 = pd.concat(df3)

In [256]:
df4 = []
df5 = []
df6 = []
for file in file_list1_t:
    data4 = load_data(file)
    temp_data4 = calculateOccupy(data4)
    temp_data4 = pd.DataFrame(temp_data4).sort_index()
    temp_data4 = temp_data4.T
    df4.append(temp_data4)
df4 = pd.concat(df4)

for file in file_list2_t:
    data5 = load_data(file)
    temp_data5 = calculateOccupy(data5)
    temp_data5 = pd.DataFrame(temp_data5).sort_index()
    temp_data5 = temp_data5.T
    df5.append(temp_data5)
df5 = pd.concat(df5)

for file in file_list3_t:
    data6 = load_data(file)
    temp_data6 = calculateOccupy(data6)
    temp_data6 = pd.DataFrame(temp_data6).sort_index()
    temp_data6 = temp_data6.T
    df6.append(temp_data6)
df6 = pd.concat(df6)

In [262]:
trace1 = go.Scatter(
    x = df1.index - 1546293600,
    y = df1['high']/(df1['high']+df1['low']),
    name = 'charge lane usage rate 1',
    line = dict(
        width = 1)
)

trace2 = go.Scatter(
    x = df2.index - 1546293600,
    y = df2['high']/(df2['high']+df2['low']),
    name = 'charge lane usage rate 2',
    line = dict(
        width = 1)
)

trace3 = go.Scatter(
    x = df3.index - 1546293600,
    y = df3['high']/(df3['high']+df3['low']),
    name = 'charge lane usage rate 3',
    line = dict(
        width = 1)
)


trace4 = go.Scatter(
    x = df4.index - 1546293600,
    y = df4['high']/(df4['high']+df4['low']),
    name = 'charge lane usage rate 1_t',
    line = dict(
        width = 1, dash = "dot")
)

trace5 = go.Scatter(
    x = df5.index - 1546293600,
    y = df5['high']/(df5['high']+df5['low']),
    name = 'charge lane usage rate 1_t',
    line = dict(
        width = 1, dash = "dot")
)

trace6 = go.Scatter(
    x = df6.index - 1546293600,
    y = df6['high']/(df6['high']+df6['low']),
    name = 'charge lane usage rate 1_t',
    line = dict(
        width = 1, dash = "dot")
)

# format
data = [trace1, trace2, trace3, trace4, trace5, trace6]

layout = go.Layout(
    title=go.layout.Title(
        text='Charge Lane Usage Rate (normal sim)',
        xref='paper',
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='time',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='usage',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename = 'Charge Lane Usage Rate (normal sim)', auto_open=True)

The draw time for this plot will be slow for clients without much RAM.


C:\Users\liyanyan\AppData\Local\Continuum\anaconda3\lib\site-packages\chart_studio\api\v1\clientresp.py:44: UserWarning:

Estimated Draw Time Slow



'https://plot.ly/~lyy901207/68'

In [76]:
# pcu plot
# create a figure showing the the highest PCU and mean PCU for the whole netwrok for each timestamp

# extract data
df1 = []
df2 = []
df3 = []
for file in file_list1:
    data1 = load_data(file)
    temp_data1 = extract_data(data1, 'lanes', 'countPcu')
    temp_data1 = pd.DataFrame(temp_data1).sort_index()
    temp_data1 = temp_data1.T
    df1.append(temp_data1)
df1 = pd.concat(df1)

for file in file_list2:
    data2 = load_data(file)
    temp_data2 = extract_data(data2, 'lanes', 'countPcu')
    temp_data2 = pd.DataFrame(temp_data2).sort_index()
    temp_data2 = temp_data2.T
    df2.append(temp_data2)
df2 = pd.concat(df2)

for file in file_list3:
    data3 = load_data(file)
    temp_data3 = extract_data(data3, 'lanes', 'countPcu')
    temp_data3 = pd.DataFrame(temp_data3).sort_index()
    temp_data3 = temp_data3.T
    df3.append(temp_data3)
df3 = pd.concat(df3)

# write csv (optimal)
#write_csv(df,'C:/Users/lyy90/OneDrive/Documents/GitHub/meso_v2.0/visualization/uniform_fix2_pcu.csv')



KeyboardInterrupt: 

In [55]:
df4 = []
df5 = []
df6 = []
for file in file_list1_t:
    data4 = load_data(file)
    temp_data4 = extract_data(data4, 'lanes', 'countPcu')
    temp_data4 = pd.DataFrame(temp_data4).sort_index()
    temp_data4 = temp_data1.T
    df4.append(temp_data4)
df4 = pd.concat(df4)

for file in file_list2_t:
    data5 = load_data(file)
    temp_data5 = extract_data(data5, 'lanes', 'countPcu')
    temp_data5 = pd.DataFrame(temp_data5).sort_index()
    temp_data5 = temp_data5.T
    df5.append(temp_data5)
df5 = pd.concat(df5)

for file in file_list3_t:
    data6 = load_data(file)
    temp_data6 = extract_data(data6, 'lanes', 'countPcu')
    temp_data6 = pd.DataFrame(temp_data6).sort_index()
    temp_data6 = temp_data6.T
    df6.append(temp_data6)
df6 = pd.concat(df6)

In [56]:
# max pcu
trace0 = go.Scatter(
    #x=df.index,
    x = df1.index - 1546293600,
    y = df1.max(axis =1),
    name = 'max_pcu 1',
    line = dict(
        width = 1)
)

# mean pcu
trace1 = go.Scatter(
    x = df1.index - 1546293600,
    y = df1.mean(axis =1),
    name = 'mean_pcu 1',
    line = dict(
        width = 1, dash = 'dot')
)

trace2 = go.Scatter(
    x = df2.index - 1546293600,
    y = df2.mean(axis =1),
    name = 'max_pcu 2',
    line = dict(
        width = 1)
)

trace3 = go.Scatter(
    x = df1.index - 1546293600,
    y = df1.mean(axis =1),
    name = 'mean_pcu 2',
    line = dict(
        width = 1, dash = 'dot')
)

trace4 = go.Scatter(
    x = df3.index - 1546293600,
    y = df3.max(axis =1),
    name = 'max_pcu 3',
    line = dict(
        width = 1)
)

trace5 = go.Scatter(
    x = df3.index - 1546293600,
    y = df3.mean(axis =1),
    name = 'mean_pcu 3',
    line = dict(
        width = 1,dash = 'dot')
)
# format
data = [trace0, trace1, trace2, trace3, trace4, trace5]

layout = go.Layout(
    title=go.layout.Title(
        text='Count PCU for Lanes (normal_sim)',
        xref='paper',
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='time',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='pcu',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename = 'pcu (normal_sim)', auto_open=True)

'https://plot.ly/~lyy901207/36'

In [24]:
# Speed plot
# create a figure showing the the highest speed, lowest speed and mean PCU for the whole netwrok for each timestamp

# extract data
df1 = []
df2 = []
df3 = []
for file in file_list1:
    data1 = load_data(file)
    temp_data1 = extract_data(data1, 'lanes', 'speed')
    temp_data1 = pd.DataFrame(temp_data1).sort_index()
    temp_data1 = temp_data1.T
    df1.append(temp_data1)
df1 = pd.concat(df1)

for file in file_list2:
    data2 = load_data(file)
    temp_data2 = extract_data(data2, 'lanes', 'speed')
    temp_data2 = pd.DataFrame(temp_data2).sort_index()
    temp_data2 = temp_data2.T
    df2.append(temp_data2)
df2 = pd.concat(df2)

for file in file_list3:
    data3 = load_data(file)
    temp_data3 = extract_data(data3, 'lanes', 'speed')
    temp_data3 = pd.DataFrame(temp_data3).sort_index()
    temp_data3 = temp_data3.T
    df3.append(temp_data3)
df3 = pd.concat(df3)

# write csv (optimal)
#write_csv(df,'C:/Users/lyy90/OneDrive/Documents/GitHub/meso_v2.0/visualization/uniform_fix2_pcu.csv')



In [25]:
df4 = []
df5 = []
df6 = []
for file in file_list1_t:
    data4 = load_data(file)
    temp_data4 = extract_data(data4, 'lanes', 'speed')
    temp_data4 = pd.DataFrame(temp_data4).sort_index()
    temp_data4 = temp_data4.T
    df4.append(temp_data4)
df4 = pd.concat(df4)

for file in file_list2_t:
    data5 = load_data(file)
    temp_data5 = extract_data(data5, 'lanes', 'speed')
    temp_data5 = pd.DataFrame(temp_data5).sort_index()
    temp_data5 = temp_data5.T
    df5.append(temp_data5)
df5 = pd.concat(df5)

for file in file_list3_t:
    data6 = load_data(file)
    temp_data6 = extract_data(data6, 'lanes', 'speed')
    temp_data6 = pd.DataFrame(temp_data6).sort_index()
    temp_data6 = temp_data6.T
    df6.append(temp_data6)
df6 = pd.concat(df6)

In [26]:
# mean speed
trace0 = go.Scatter(
    #x=df.index,
    x = df1.index - 1546293600,
    y=df1.mean(axis =1),
    name = 'mean speed 1',
    line = dict(
        width = 1)
)

# mean pcu
trace1 = go.Scatter(
    x=df2.index - 1546293600,
    y=df2.mean(axis =1),
    name = 'mean speed 2',
    line = dict(
        width = 1)
)

trace2 = go.Scatter(
    #x=df.index,
    x = df3.index - 1546293600,
    y=df3.mean(axis =1),
    name = 'mean speed 3',
    line = dict(
        width = 1)
)

# mean pcu
trace3 = go.Scatter(
    x=df4.index - 1546293600,
    y=df4.mean(axis =1),
    name = 'mean speed 1_t',
    line = dict(
        width = 1, dash = 'dot')
)

trace4 = go.Scatter(
    #x=df.index,
    x = df5.index - 1546293600,
    y=df5.mean(axis =1),
    name = 'mean speed 2_t',
    line = dict(
        width = 1, dash = 'dot')
)

# mean pcu
trace5 = go.Scatter(
    x=df6.index - 1546293600,
    y=df6.mean(axis =1),
    name = 'mean speed 3_t',
    line = dict(
        width = 1, dash = 'dot')
)

data = [trace0, trace1, trace2, trace3, trace4, trace5]

layout = go.Layout(
    title=go.layout.Title(
        text='Speed for Lanes (normal sim)',
        xref='paper',
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='time',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='speed',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename = 'Speed (normal sim)', auto_open=True)

'https://plot.ly/~lyy901207/118'

In [ ]:
# count PCU

In [ ]:
df4 = []
df5 = []
df6 = []
for file in file_list1_t:
    data4 = load_data(file)
    temp_data4 = extract_data(data4, 'lanes', 'countPcu')
    temp_data4 = pd.DataFrame(temp_data4).sort_index()
    temp_data4 = temp_data1.T
    df4.append(temp_data4)
df4 = pd.concat(df4)

for file in file_list2_t:
    data5 = load_data(file)
    temp_data5 = extract_data(data5, 'lanes', 'countPcu')
    temp_data5 = pd.DataFrame(temp_data5).sort_index()
    temp_data5 = temp_data5.T
    df5.append(temp_data5)
df5 = pd.concat(df5)

for file in file_list3_t:
    data6 = load_data(file)
    temp_data6 = extract_data(data6, 'lanes', 'countPcu')
    temp_data6 = pd.DataFrame(temp_data6).sort_index()
    temp_data6 = temp_data6.T
    df6.append(temp_data6)
df6 = pd.concat(df6)

In [ ]:
# min speed
trace0 = go.Scatter(
    #x=df.index,
    x = df.index - 1546293600,
    y=df.min(axis =1),
    name = 'low_speed 1',
    line = dict(
        width = 1)
)

# mean pcu
trace1 = go.Scatter(
    x=df1.index - 1546293600,
    y=df1.mean(axis =1),
    name = 'mean_speed 1',
    line = dict(
        width = 1, dash = 'dot')
)

trace2 = go.Scatter(
    #x=df.index,
    x = df2.index - 1546293600,
    y=df2.min(axis =1),
    name = 'low_speed 2',
    line = dict(
        width = 1)
)

# mean pcu
trace3 = go.Scatter(
    x=df2.index - 1546293600,
    y=df2.mean(axis =1),
    name = 'mean_speed 2',
    line = dict(
        width = 1, dash = 'dot')
)

trace4 = go.Scatter(
    #x=df.index,
    x = df3.index - 1546293600,
    y=df3.min(axis =1),
    name = 'low_speed 3',
    line = dict(
        width = 1)
)

# mean pcu
trace5 = go.Scatter(
    x=df3.index - 1546293600,
    y=df3.mean(axis =1),
    name = 'mean_speed 3',
    line = dict(
        width = 1, dash = 'dot')
)

data = [trace0, trace1, trace2, trace3, trace4, trace5]

layout = go.Layout(
    title=go.layout.Title(
        text='Speed for Lanes (uniform_fix)',
        xref='paper',
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='time',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='speed',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
)


fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename = 'Speed (uniform_fix)', auto_open=True)